In [2]:
"""为序数特征选择算法"""

'为序数特征选择算法'

In [6]:
"""
    降维：
        特征选择：从原始特征中选取子集
            原理：自动选择与问题最相关的特征子集，提高模型的计算效率或者通过去除无关特征消除噪声降低模型的泛化误差
        特征提取：从特征集中提取信息以构造新的特征子空间
    
    顺序特征选择属于贪婪搜索算法
    经典的序数特征选择算法是逆顺序选择（SBS）
        目的：应对分类器性能最小的衰减来降低初始特征子空间的维数，从而提高计算效率
        逻辑：顺序的从完整的特征子集中移除特征，知道新特征子空间包含需要的特征数量。
        
    SBS算法：
        1.用k=d初始化算法，d为特征空间Xd的维数
        2.确定使标准值最大的特征x-，x-=argmaxJ(Xk-x),其中x属于Xk
        3.从特征集中去处特征x-；Xk = Xk - x-；k = k -1；
        4 如果k等于期望的特征数则停止，否则转回2
"""

'\n    降维：\n        特征选择：从原始特征中选取子集\n            原理：自动选择与问题最相关的特征子集，提高模型的计算效率或者通过去除无关特征消除噪声降低模型的泛化误差\n        特征提取：从特征集中提取信息以构造新的特征子空间\n    \n    顺序特征选择属于贪婪搜索算法\n    经典的序数特征选择算法是逆顺序选择（SBS）\n        目的：应对分类器性能最小的衰减来降低初始特征子空间的维数，从而提高计算效率\n        逻辑：顺序的从完整的特征子集中移除特征，知道新特征子空间包含需要的特征数量。\n        \n    SBS算法：\n        1.用k=d初始化算法，d为特征空间Xd的维数\n        2.确定使标准值最大的特征x-，x-=argmaxJ(Xk-x),其中x属于Xk\n        3.从特征集中去处特征x-；Xk = Xk - x-；k = k -1；\n        4 如果k等于期望的特征数则停止，否则转回2\n'

In [7]:
from sklearn.base import clone

In [8]:
from itertools import combinations

In [9]:
import numpy as np

In [10]:
from sklearn.metrics import accuracy_score

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
class SBS():
    def __init__(self, estimator, k_features,
                scoring=accuracy_score,
                test_size=0.25, random_state=1):
        self.scoring = scoring
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state
        
    def fit(self, X, y):
        X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=self.test_size,
                        random_state=self.random_state)
        
        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))
        self.subsets_ = [self.indices_]
        score = self._calc_score(X_train, y_train,
                                 X_test, y_test, self.indices_)

        self.scores_ = [score]

        while dim > self.k_features:
            scores = []
            subsets  = []

            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train,
                                        X_test, y_test, p)
                scores.append(score)
                subsets.append(p)

            best = np.argmax(scores)
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1

            self.scores_.append(scores[best])
            self.k_score_ = self.scores_[-1]
            
        return self
    
    def transform(self, X):
        return X[:, self.indices_]
    
    def _cale_score(self, X_train, y_train, X_test, y_test,
                   indices):
        self.estimator.fit(X_train[:, indices], y_train)
        y_pred = self.estimator.predict(X_test[:, indices])
        score = self.scoring(y_test, y_pred)
        return score

In [13]:
import matplotlib.pyplot as plt

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
knn = KNeighborsClassifier(n_neighbors=5)

In [16]:
sbs = SBS(knn, k_features=1)

In [17]:
sbs.fit(X_train_std, y_train)

NameError: name 'X_train_std' is not defined